In [1]:
import requests
import re
from bs4 import BeautifulSoup
import os
import threading
import time

In [2]:
pattern = re.compile('\s*')

In [3]:
def give_MGyear_2_responce(MG_year):
    year = 1911 + MG_year
    url = 'https://www.ceec.edu.tw/xmfile?xsmsid=0J052424829869345634'
    data = {
        '__RequestVerificationToken': '-lfI9abPMDVb6eACo_9WXhCIiZwGR4GeCHlw1wKW7lG8z7oQsj_74cpoWZYSk5JF2qvYanNxR_WB5vIZAmI3bA6Zz_AED2dI_YSfuQhSpC41',
        'Annaul': f'{year}',
    }
    headers = {
        'Cookie': '__RequestVerificationToken=_l0MnYxqpEVljEZM61zwB_Zr0B6SmvCjARGNAjgz1op0ibIiCZSuAdQbo_jgJKGTcFEBN9fZPOHk0Diy3iIO6i83uueLhY3EfE4x1A4Nzqg1; FSize=M; _ga=GA1.3.1420122771.1644909938; _gid=GA1.3.114032808.1644909938; TS01a20b7e=011a5189c9c84c3c9e7d42579632fc7a307209e8b6fc96431d415582a1403e7a62d6afed4224662c62a33b1f0864a3e7a868a8a417948275b148cd2e72f31fd1cd13d57118; _gat=1',
        'Origin': 'https://www.ceec.edu.tw',
        }
    try:
        response = requests.post(url,data=data,headers=headers)
        print(f'{MG_year}\t{response.status_code}')
        return response
    except Exception as e:
        print(e)

In [4]:
response = give_MGyear_2_responce(83)

83	200


In [5]:
bsobj = BeautifulSoup(response.text) # 解析
table = bsobj.find('table',attrs={'class':'rwdTable'}) # 取得 表格

In [6]:
def get_attrs(exam_class_set, exam_class_name, MG_year): # 對表格中的考科的一個下載按鈕取出"連結"、"檔名"
    # 回傳連結與將要儲存的檔案位置
    website = 'https://www.ceec.edu.tw'
    href = exam_class_set.attrs['href']
    if website not in href:
        href=website+href
    path = os.path.join(str(MG_year),exam_class_name, exam_class_set.attrs['title']+'.pdf')
    return [href,path,]

In [7]:
def one_year_table2download_dict(table,MG_year):
    # exam_class: 表格中的一個考科、 wanted: 要下載的元素
    one_year_need_download = {}
    for exam_class in table.find_all('tr')[1:]: 
        wanted = [exam_class.find_all('a')[0], 
                  exam_class.find_all('a')[2]]
        exam_class_name = pattern.sub('',exam_class.find('td',attrs={'class':'title'}).text) # 考科資料夾名稱
        exam_class_download_list = [get_attrs(i, exam_class_name, MG_year) for i in wanted]
        one_year_need_download.update({exam_class_name: exam_class_download_list})
    return one_year_need_download

In [8]:
if not os.path.isdir('download'):
    os.mkdir('download')
for MG_year in range(83, 110):
    dir_path = os.path.join('download', str(MG_year))
    if not os.path.isdir(dir_path):
        os.mkdir(dir_path)

In [9]:
def give_MGyear_get_download_list(MG_year):
    response = give_MGyear_2_responce(MG_year)
    bsobj = BeautifulSoup(response.text) # 解析
    table = bsobj.find('table',attrs={'class':'rwdTable'}) # 取得 表格
    one_year_need_download = one_year_table2download_dict(table, MG_year)
    download_list = []
    for exam_class_name in one_year_need_download:
        download_list+=(one_year_need_download[exam_class_name]) # 每個元素都是[url, file path]
    return download_list

In [10]:
def download_and_save(url, save_path):
    response = requests.get(url)
    save_path = os.path.join('download' ,save_path) # download\\110\\110學年度學科能力測驗－自然\\110學測自然答案.pdf
    mother_dir = os.path.join(*save_path.split('\\')[:-1]) # download\\110\\110學年度學科能力測驗－自然
    if not os.path.isdir(mother_dir):
        os.mkdir(mother_dir)
    with open(save_path, 'wb') as f:
        f.write(response.content)

In [11]:
def given_MG_year_download_save(MG_year):
    download_list = give_MGyear_get_download_list(MG_year)
    for url, save_path in download_list:
        download_and_save(url, save_path)

In [12]:
class Thread4Year(threading.Thread):
    def __init__(self,func,args=()):
        super(Thread4Year,self).__init__()
        self.func = func
        self.args = args
    def run(self):
        self.result = self.func(*self.args)
    def get_result(self):
        try:
            return self.result
        except Exception:
            return None

In [13]:
threads_list = [Thread4Year(given_MG_year_download_save, (i,)) for i in range(83,110)]

In [15]:
start = time.time()
for thread in threads_list:
    thread.start()
for thread in threads_list:
    thread.join() 
end = time.time()
print(f'Cost {end-start:.2f} seconds')

83	200
87	200
88	200
84	200
90	200
85	200
86	200
91	200
94	200
101	200
97	200
93	200
98	200
102	200
89	200
106	200
104	200
96	200
100	200
95	200
105	200
99	200
92	200
107	200
108	200
109	200
103	200
Cost 18.37 seconds
